# E-commerce Business Analytics Dashboard

**Comprehensive analysis of e-commerce performance metrics and trends**

---

## Table of Contents

1. [Introduction & Business Objectives](#introduction)
2. [Data Loading & Configuration](#data-loading)
3. [Data Dictionary](#data-dictionary)
4. [Data Preparation & Transformation](#data-preparation)
5. [Business Metrics Calculation](#business-metrics)
   - [Revenue Analysis](#revenue-analysis)
   - [Product Performance Analysis](#product-analysis)
   - [Geographic Analysis](#geographic-analysis)
   - [Customer Experience Analysis](#customer-experience)
6. [Summary of Key Observations](#summary)

---

## 1. Introduction & Business Objectives {#introduction}

This notebook provides a comprehensive analysis of e-commerce business performance, focusing on key metrics that drive business decisions:

**Primary Business Objectives:**
- Analyze revenue trends and growth patterns
- Identify top-performing product categories and geographic markets
- Evaluate customer experience through delivery performance and satisfaction metrics
- Assess operational efficiency and fulfillment performance

**Key Questions Addressed:**
- How has revenue performance changed over time?
- Which product categories and geographic regions drive the most revenue?
- What is the relationship between delivery performance and customer satisfaction?
- How efficient are our order fulfillment processes?

**Analysis Framework:**
This analysis uses a configurable date range approach, allowing for flexible period comparisons and trend analysis.

## 2. Data Loading & Configuration {#data-loading}

Configure analysis parameters and load the required datasets.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from data_loader import EcommerceDataLoader, load_and_prepare_data
from business_metrics import BusinessMetricsCalculator, calculate_period_comparison

# Configure visualization settings
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries and modules loaded successfully")

In [ ]:
# Analysis Configuration
ANALYSIS_YEAR = 2023
COMPARISON_YEAR = 2022
DATA_PATH = '../ecommerce_data/'

print(f"Analysis Configuration:")
print(f"- Primary Analysis Year: {ANALYSIS_YEAR}")
print(f"- Comparison Year: {COMPARISON_YEAR}")
print(f"- Data Path: {DATA_PATH}")

In [ ]:
# Load and prepare data for analysis
print("Loading data for analysis year...")
current_data, loader = load_and_prepare_data(DATA_PATH, year=ANALYSIS_YEAR)

print("\nLoading data for comparison year...")
comparison_data, _ = load_and_prepare_data(DATA_PATH, year=COMPARISON_YEAR)

print(f"\nData Loading Summary:")
print(f"- {ANALYSIS_YEAR} Dataset: {len(current_data):,} records")
print(f"- {COMPARISON_YEAR} Dataset: {len(comparison_data):,} records")
print(f"- Date Range ({ANALYSIS_YEAR}): {current_data['order_purchase_timestamp'].min().date()} to {current_data['order_purchase_timestamp'].max().date()}")

## 3. Data Dictionary {#data-dictionary}

**Key Business Terms and Data Definitions:**

### Order Information
- **Order ID**: Unique identifier for each customer order
- **Order Status**: Current status (delivered, canceled, pending, processing, shipped, returned)
- **Order Purchase Timestamp**: Date and time when order was placed
- **Delivery Days**: Time between order placement and customer delivery

### Financial Metrics
- **Revenue**: Total sales value from delivered orders (excludes canceled/returned)
- **Average Order Value (AOV)**: Average total value per order
- **Price**: Individual item price (excluding freight)
- **Total Item Value**: Item price plus freight costs

### Product Information
- **Product Category**: Standardized product category names
- **Category Clean**: Human-readable category names (spaces, title case)

### Customer Data
- **Customer State**: Two-letter state code for customer location
- **Customer City**: Customer's city location

### Performance Metrics
- **Review Score**: Customer satisfaction rating (1-5 scale)
- **Fulfillment Rate**: Percentage of orders successfully delivered
- **Cancellation Rate**: Percentage of orders canceled
- **Return Rate**: Percentage of delivered orders that were returned

## 4. Data Preparation & Transformation {#data-preparation}

Examine data quality and prepare datasets for analysis.

In [ ]:
# Data Quality Assessment
print("Data Quality Summary for Analysis Year:")
print("=" * 50)

# Basic dataset information
print(f"Total Records: {len(current_data):,}")
print(f"Date Range: {current_data['order_purchase_timestamp'].min().date()} to {current_data['order_purchase_timestamp'].max().date()}")
print(f"Unique Orders: {current_data['order_id'].nunique():,}")
print(f"Unique Customers: {current_data['customer_id'].nunique():,}")
print(f"Unique Products: {current_data['product_id'].nunique():,}")

# Order status distribution
print("\nOrder Status Distribution:")
status_counts = current_data['order_status'].value_counts()
for status, count in status_counts.items():
    percentage = (count / len(current_data)) * 100
    print(f"  {status.title()}: {count:,} ({percentage:.1f}%)")

# Missing data analysis
print("\nMissing Data Analysis:")
missing_data = current_data.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
if len(missing_data) > 0:
    for column, missing_count in missing_data.items():
        percentage = (missing_count / len(current_data)) * 100
        print(f"  {column}: {missing_count:,} ({percentage:.1f}%)")
else:
    print("  No missing data found")

In [ ]:
# Filter data for delivered orders (primary analysis dataset)
delivered_current = current_data[current_data['order_status'] == 'delivered'].copy()
delivered_comparison = comparison_data[comparison_data['order_status'] == 'delivered'].copy()

print(f"Delivered Orders Analysis Dataset:")
print(f"- {ANALYSIS_YEAR}: {len(delivered_current):,} delivered orders")
print(f"- {COMPARISON_YEAR}: {len(delivered_comparison):,} delivered orders")
print(f"- Year-over-year change: {len(delivered_current) - len(delivered_comparison):,} orders")

# Revenue summary
current_revenue = delivered_current['price'].sum()
comparison_revenue = delivered_comparison['price'].sum()
revenue_growth = ((current_revenue - comparison_revenue) / comparison_revenue) * 100

print(f"\nRevenue Summary:")
print(f"- {ANALYSIS_YEAR} Revenue: ${current_revenue:,.2f}")
print(f"- {COMPARISON_YEAR} Revenue: ${comparison_revenue:,.2f}")
print(f"- Year-over-year growth: {revenue_growth:+.1f}%")

## 5. Business Metrics Calculation {#business-metrics}

Calculate comprehensive business metrics across key performance areas.

In [ ]:
# Initialize business metrics calculators
current_calculator = BusinessMetricsCalculator(current_data)
comparison_calculator = BusinessMetricsCalculator(comparison_data)

# Generate executive summary
executive_summary = current_calculator.generate_executive_summary(comparison_data)

print("Executive Summary")
print("=" * 50)
print(f"Total Revenue: ${executive_summary['period_summary']['total_revenue']:,.2f}")
print(f"Total Orders: {executive_summary['period_summary']['total_orders']:,}")
print(f"Average Order Value: ${executive_summary['period_summary']['average_order_value']:.2f}")
print(f"Fulfillment Rate: {executive_summary['period_summary']['fulfillment_rate']:.1f}%")

print(f"\nGrowth Metrics:")
print(f"Revenue Growth: {executive_summary['growth_metrics']['revenue_growth']:+.1f}%")
print(f"Order Growth: {executive_summary['growth_metrics']['order_growth']:+.1f}%")
print(f"AOV Growth: {executive_summary['growth_metrics']['aov_growth']:+.1f}%")

### 5.1 Revenue Analysis {#revenue-analysis}

Comprehensive analysis of revenue trends, growth patterns, and key performance indicators.

In [ ]:
# Calculate detailed revenue metrics
revenue_metrics = current_calculator.calculate_revenue_metrics(comparison_data)

print("Revenue Analysis")
print("=" * 40)
print(f"Total Revenue: ${revenue_metrics['total_revenue']:,.2f}")
print(f"Total Orders: {revenue_metrics['total_orders']:,}")
print(f"Total Items Sold: {revenue_metrics['total_items_sold']:,}")
print(f"Average Order Value: ${revenue_metrics['average_order_value']:.2f}")
print(f"Average Item Price: ${revenue_metrics['average_item_price']:.2f}")

if 'revenue_growth' in revenue_metrics:
    print(f"\nYear-over-Year Growth:")
    print(f"Revenue Growth: {revenue_metrics['revenue_growth']:+.1f}%")
    print(f"Order Growth: {revenue_metrics['order_growth']:+.1f}%")
    print(f"AOV Growth: {revenue_metrics['aov_growth']:+.1f}%")

In [ ]:
# Monthly revenue trend visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Current year monthly trend
monthly_trend = revenue_metrics['monthly_revenue_trend']
ax1.plot(monthly_trend['order_month'], monthly_trend['price'], 
         marker='o', linewidth=2, markersize=6, color='#2E86AB')
ax1.set_title(f'Monthly Revenue Trend - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
ax1.set_xlabel('Month')
ax1.set_ylabel('Revenue ($)')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='y', rotation=0)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Year-over-year comparison
comparison_monthly = comparison_calculator.calculate_revenue_metrics()['monthly_revenue_trend']
ax2.plot(monthly_trend['order_month'], monthly_trend['price'], 
         marker='o', linewidth=2, label=f'{ANALYSIS_YEAR}', color='#2E86AB')
ax2.plot(comparison_monthly['order_month'], comparison_monthly['price'], 
         marker='s', linewidth=2, linestyle='--', label=f'{COMPARISON_YEAR}', color='#A23B72')
ax2.set_title(f'Revenue Comparison: {ANALYSIS_YEAR} vs {COMPARISON_YEAR}', fontsize=14, fontweight='bold')
ax2.set_xlabel('Month')
ax2.set_ylabel('Revenue ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

plt.tight_layout()
plt.show()

### 5.2 Product Performance Analysis {#product-analysis}

Analysis of product categories, top performers, and market share distribution.

In [ ]:
# Calculate product performance metrics
product_metrics = current_calculator.calculate_product_metrics()

print("Product Performance Summary")
print("=" * 40)
print(f"Total Product Categories: {product_metrics['total_categories']}")
print(f"Total Unique Products: {product_metrics['total_products']:,}")

print("\nTop 5 Categories by Revenue:")
top_categories = product_metrics['category_metrics'].head()
for idx, (category, row) in enumerate(top_categories.iterrows(), 1):
    print(f"{idx}. {category}: ${row['total_revenue']:,.2f} ({row['revenue_share']:.1f}% of total)")

In [ ]:
# Product category performance visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Top 10 categories by revenue
top_10_categories = product_metrics['category_metrics'].head(10)
colors = plt.cm.Blues(np.linspace(0.4, 0.8, len(top_10_categories)))

bars = ax1.barh(range(len(top_10_categories)), top_10_categories['total_revenue'], color=colors)
ax1.set_yticks(range(len(top_10_categories)))
ax1.set_yticklabels(top_10_categories.index, fontsize=10)
ax1.set_xlabel('Revenue ($)')
ax1.set_title(f'Top 10 Product Categories by Revenue - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Add value labels on bars
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax1.text(width + width*0.01, bar.get_y() + bar.get_height()/2, 
             f'${width/1000:.0f}K', ha='left', va='center', fontsize=9)

# Revenue share pie chart
top_5_categories = product_metrics['category_metrics'].head(5)
other_revenue = product_metrics['category_metrics'].iloc[5:]['total_revenue'].sum()

pie_data = list(top_5_categories['total_revenue']) + [other_revenue]
pie_labels = list(top_5_categories.index) + ['Others']
colors_pie = plt.cm.Set3(np.linspace(0, 1, len(pie_data)))

wedges, texts, autotexts = ax2.pie(pie_data, labels=pie_labels, autopct='%1.1f%%', 
                                   colors=colors_pie, startangle=90)
ax2.set_title(f'Revenue Share by Category - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 5.3 Geographic Analysis {#geographic-analysis}

Analysis of revenue distribution across states and cities, identifying key geographic markets.

In [ ]:
# Calculate geographic performance metrics
geographic_metrics = current_calculator.calculate_geographic_metrics()

print("Geographic Performance Summary")
print("=" * 40)
print(f"Total States: {geographic_metrics['total_states']}")
print(f"Total Cities: {geographic_metrics['total_cities']:,}")

print("\nTop 5 States by Revenue:")
top_states = geographic_metrics['state_metrics'].head()
for idx, (state, row) in enumerate(top_states.iterrows(), 1):
    print(f"{idx}. {state}: ${row['total_revenue']:,.2f} ({row['revenue_share']:.1f}% of total)")

print("\nTop 5 Cities by Revenue:")
top_cities = geographic_metrics['top_cities'].head()
for idx, row in top_cities.iterrows():
    print(f"{idx+1}. {row['customer_city']}, {row['customer_state']}: ${row['price']:,.2f}")

In [ ]:
# Geographic performance visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))

# Top 15 states by revenue
top_15_states = geographic_metrics['state_metrics'].head(15)
colors = plt.cm.Greens(np.linspace(0.4, 0.8, len(top_15_states)))

bars = ax1.bar(range(len(top_15_states)), top_15_states['total_revenue'], color=colors)
ax1.set_xticks(range(len(top_15_states)))
ax1.set_xticklabels(top_15_states.index, rotation=45, ha='right')
ax1.set_ylabel('Revenue ($)')
ax1.set_title(f'Top 15 States by Revenue - {ANALYSIS_YEAR}', fontsize=14, fontweight='bold')
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'${height/1000:.0f}K', ha='center', va='bottom', fontsize=8, rotation=90)

# Revenue vs Order Count scatter plot
scatter_data = geographic_metrics['state_metrics'].head(20)
scatter = ax2.scatter(scatter_data['orders'], scatter_data['total_revenue'], 
                     s=100, alpha=0.6, c=scatter_data['avg_order_value'], 
                     cmap='viridis')
ax2.set_xlabel('Number of Orders')
ax2.set_ylabel('Total Revenue ($)')
ax2.set_title(f'Revenue vs Order Volume by State - {ANALYSIS_YEAR}\n(Color indicates Average Order Value)', 
              fontsize=14, fontweight='bold')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
ax2.grid(True, alpha=0.3)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax2)
cbar.set_label('Average Order Value ($)')

# Annotate top 5 states
for idx, (state, row) in enumerate(scatter_data.head(5).iterrows()):
    ax2.annotate(state, (row['orders'], row['total_revenue']), 
                xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.tight_layout()
plt.show()

### 5.4 Customer Experience Analysis {#customer-experience}

Analysis of delivery performance, customer satisfaction, and the relationship between service quality and customer reviews.

In [ ]:
# Calculate customer experience metrics
cx_metrics = current_calculator.calculate_customer_experience_metrics()
operational_metrics = current_calculator.calculate_operational_metrics()

print("Customer Experience Summary")
print("=" * 40)

# Delivery performance
delivery_metrics = cx_metrics['delivery_metrics']
print(f"Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
print(f"Median Delivery Time: {delivery_metrics['median_delivery_days']:.1f} days")

# Review metrics
review_metrics = cx_metrics['review_metrics']
print(f"\nCustomer Satisfaction:")
print(f"Average Review Score: {review_metrics['avg_review_score']:.2f}/5.0")
print(f"Total Reviews: {review_metrics['total_reviews']:,}")
print(f"Review Rate: {review_metrics['review_rate']:.1f}%")

# Operational metrics
print(f"\nOperational Performance:")
print(f"Fulfillment Rate: {operational_metrics['fulfillment_rate']:.1f}%")
print(f"Cancellation Rate: {operational_metrics['cancellation_rate']:.1f}%")
print(f"Return Rate: {operational_metrics['return_rate']:.1f}%")

In [ ]:
# Customer experience visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Delivery time distribution
delivery_dist = cx_metrics['delivery_distribution']
colors_delivery = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(delivery_dist)))
bars1 = ax1.bar(range(len(delivery_dist)), delivery_dist.values, color=colors_delivery)
ax1.set_xticks(range(len(delivery_dist)))
ax1.set_xticklabels(delivery_dist.index, rotation=45)
ax1.set_ylabel('Number of Orders')
ax1.set_title(f'Delivery Time Distribution - {ANALYSIS_YEAR}', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{int(height):,}', ha='center', va='bottom', fontsize=9)

# Review score distribution
review_dist = review_metrics['review_distribution']
colors_review = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(review_dist)))
bars2 = ax2.bar(review_dist.index, review_dist.values, color=colors_review)
ax2.set_xlabel('Review Score')
ax2.set_ylabel('Number of Reviews')
ax2.set_title(f'Customer Review Score Distribution - {ANALYSIS_YEAR}', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{int(height):,}', ha='center', va='bottom', fontsize=9)

# Satisfaction by delivery time
if not cx_metrics['satisfaction_by_delivery'].empty:
    satisfaction_data = cx_metrics['satisfaction_by_delivery']
    bars3 = ax3.bar(range(len(satisfaction_data)), satisfaction_data['avg_review_score'], 
                    color='#FF6B6B', alpha=0.7)
    ax3.set_xticks(range(len(satisfaction_data)))
    ax3.set_xticklabels(satisfaction_data.index, rotation=45)
    ax3.set_ylabel('Average Review Score')
    ax3.set_title(f'Customer Satisfaction vs Delivery Time - {ANALYSIS_YEAR}', fontsize=12, fontweight='bold')
    ax3.set_ylim(0, 5)
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar in bars3:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                 f'{height:.2f}', ha='center', va='bottom', fontsize=9)
else:
    ax3.text(0.5, 0.5, 'Insufficient data for\nsatisfaction analysis', 
             ha='center', va='center', transform=ax3.transAxes, fontsize=12)
    ax3.set_title('Customer Satisfaction vs Delivery Time', fontsize=12, fontweight='bold')

# Order status distribution
status_data = operational_metrics['order_status_percentages']
colors_status = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#8E44AD', '#27AE60']
wedges, texts, autotexts = ax4.pie(status_data.values, labels=status_data.index, 
                                   autopct='%1.1f%%', colors=colors_status[:len(status_data)],
                                   startangle=90)
ax4.set_title(f'Order Status Distribution - {ANALYSIS_YEAR}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Summary of Key Observations {#summary}

Based on the comprehensive analysis of the e-commerce data, here are the key findings and business insights:

In [ ]:
# Generate and display key insights
print("KEY BUSINESS INSIGHTS")
print("=" * 60)

# Revenue Performance
print("\n1. REVENUE PERFORMANCE")
print("-" * 25)
revenue_summary = executive_summary['period_summary']
growth_summary = executive_summary['growth_metrics']

print(f"• Total revenue for {ANALYSIS_YEAR}: ${revenue_summary['total_revenue']:,.2f}")
print(f"• Year-over-year revenue growth: {growth_summary['revenue_growth']:+.1f}%")
print(f"• Average order value: ${revenue_summary['average_order_value']:.2f}")
print(f"• AOV growth: {growth_summary['aov_growth']:+.1f}%")

# Product Performance
print("\n2. PRODUCT PERFORMANCE")
print("-" * 25)
top_performers = executive_summary['top_performers']
print(f"• Top performing category: {top_performers['top_category']}")
print(f"• Top category revenue: ${top_performers['top_category_revenue']:,.2f}")
print(f"• Total product categories: {product_metrics['total_categories']}")
print(f"• Total unique products: {product_metrics['total_products']:,}")

# Geographic Performance
print("\n3. GEOGRAPHIC PERFORMANCE")
print("-" * 30)
print(f"• Top performing state: {top_performers['top_state']}")
print(f"• Top state revenue: ${top_performers['top_state_revenue']:,.2f}")
print(f"• Geographic coverage: {geographic_metrics['total_states']} states, {geographic_metrics['total_cities']:,} cities")

# Customer Experience
print("\n4. CUSTOMER EXPERIENCE")
print("-" * 25)
cx_summary = executive_summary['customer_experience']
print(f"• Average delivery time: {cx_summary['avg_delivery_days']:.1f} days")
print(f"• Average customer satisfaction: {cx_summary['avg_review_score']:.2f}/5.0")
print(f"• Customer review participation: {cx_summary['review_rate']:.1f}%")

# Operational Health
print("\n5. OPERATIONAL HEALTH")
print("-" * 25)
ops_summary = executive_summary['operational_health']
print(f"• Order fulfillment rate: {ops_summary['fulfillment_rate']:.1f}%")
print(f"• Order cancellation rate: {ops_summary['cancellation_rate']:.1f}%")
print(f"• Return rate: {ops_summary['return_rate']:.1f}%")

print("\n" + "=" * 60)
print("Analysis completed successfully")
print(f"Report generated for: {ANALYSIS_YEAR} vs {COMPARISON_YEAR}")
print(f"Data processed: {len(current_data):,} total records")

### Business Recommendations

Based on the analysis findings, here are key recommendations for business improvement:

**Revenue Optimization:**
- Focus marketing efforts on high-performing product categories
- Investigate opportunities to improve average order value in underperforming segments
- Develop targeted campaigns for seasonal revenue fluctuations

**Geographic Expansion:**
- Prioritize marketing investment in top-performing states
- Analyze logistics and fulfillment capabilities in high-potential markets
- Consider regional product preferences and pricing strategies

**Customer Experience Enhancement:**
- Implement initiatives to reduce average delivery times
- Focus on maintaining high satisfaction scores while scaling operations
- Develop programs to increase customer review participation

**Operational Excellence:**
- Investigate root causes of order cancellations and returns
- Optimize fulfillment processes to maintain high delivery success rates
- Monitor operational metrics regularly for early issue detection

---

*This analysis provides a comprehensive view of business performance and can be easily adapted for different time periods or business segments by modifying the configuration parameters at the beginning of the notebook.*